
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Mechanizmy atencji Memory Network</big></big></big></big></big>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="../nn_figures/" width="100%">

# Memory Networks
MN składają się z pamięci $M$ oraz czterech funkcji
* $I$ (input feature map) konwertującej wejście na wewnętrzną reprezentację $I(x)$
  * dowolna konwersja czytelnego wejścia na cechy 
* $G$ (generalisation) modyfikującej pamięć $m_i=G(m_i, I(x))$
  * generalizacja, bo może poprawiać pamięć
  * najprostsze $G$ zapamiętuje informację w komórce $$m_{H(x)}=I(x)$$ bez modyfikacji innych
  * może być bardzo kosztowne przy dużej pamięci
    * w oryginalnej pracy pamięć miała 14 milionów komórek
    * możliwe _haszowanie_ słów w zdaniu i wstawianie do kubełka
* $O$ (output feature map) mapuje wejście i pamięć na nową reprezentację cech wyjścia $o=O(I(x), m)$
  * typowo przy odtwarzaniu odszukuje w pamięci $k$ komórek najbardziej pasujących
    * zwraca zdania porangowane
  * przy haszowaniu porównuje tylko zdania z komórek pasujących
* $R$ (response) generuje nowe wyjście z utworzonej reprezentacji $r=R(o)$
  * tworzy odpowiedź na podstawie zdania o najwyższej randze
  * gdy więcej zdań, można wykorzystać RNN dla konstrukcji odpowiedzi

## MN naiwny model podstawowy
$I, G, O, R$ muszą być nauczone: jeśli modelami dla nich są sieci neuronowe, to autorzy nazywają to __MemNN__

<img src="../nn_figures/MN.png" width="80%">

1. $I$ przetwarza tekst albo przez identyczność, albo ustalony embedding
2. $G$ zapamiętuje $I(x)$ w _następnej_ wolnej komórce
  * w wersji bardziej zaawansowanej możliwe będzie poprawianie
3. $O$ wyszukuje $k$ komórek z zawartością najbardziej spójną z pytaniem $$o_1=\arg\max_is_O(I(x), m_i)$$ gdzie $s_O$ jest odpowiednią funkcją 
  * w wersji podstawowej $k=2$
  * drugim będzie $$o_2=\arg\max_is_O([I(x),m_{o_1}], m_i)$$
4. $R$ zwraca czytelną odpowiedź
  * w najprostszej postaci to $m_{o_k}$
  * zwykle $R$ będzie rangować słowa ze znalezionych zdań $$r=\arg\max_ws_R([x,m_{o_1},m_{o_2}],w)$$
  
Tworzenie odpowiedzi wymaga __rozumienia__ słów ze słownika.

>> 
Joe went to the kitchen. Fred went to the kitchen. Joe picked up the milk.
Joe travelled to the office. Joe left the milk. Joe went to the bathroom.
* Where is the milk now? 
  * office
* Where is Joe? 
  * bathroom
* Where was Joe before the office?
  * kitchen
    

### MN uczenie
* uczenie jest w pełni nadzorowane
  * podawane pełne zdania i najlepsze odpowiedzi
* model wymaga dobrego nauczenia się funkcji $s_O$ i $s_R$
* cały model uzony SGD
  * sampling zdań

## MN rozszerzenia

### strumień słów
* słowa mogą przychodzić _nie_ jako pełne zdania, ale w strumieniu
* konieczny _dodatkowy_ model, który odkrywa fragmenty
* po znalezieniu każdego fragmentu MN traktuje ostatni jak zdanie

### efektywne wykorzystanie pamięci
* duża pamięć powoduje dużą złożoność $O$
* __kubełkowanie__
  * każde zdanie jest haszowane do kilku kubełków
  * $O$ porównuje zdanie jedynie z kubełka wyliczonego dla zapytanie
    * haszowanie na poziomie słów
      * tylko zdania dzielące słowa będą sprawdzane
    * klastrowanie embeddingów słów
      * embeddingi pozwalają na sprawdzanie także synonimów

### relacja w czasie
* wykorzystanie $\Phi(x,y,y')$ określającą relację w czasie
* ułatwia wyszukiwanie odpowiedzi

### modelowanie nie widzianych słów
* czasem pojawiają się nazwy własne i inne słowa widziane po raz pierwszy
* model językowy wykorzystujący lewy i prawy kontekst rozpoznaje czym nowe słowo powinno być
  * dla każdego słowa pamiętany zbiór jego lewych i prawych kontekstów
  * w trakcie uczenia algorytm losowo "udaje", że słowa nie widział wcześniej douczając się
  
### wiedza o świecie
* poza zdaniami o konkretnej sytuacji model może otrzymywać ogólną wiedzę o świecie

>>
* Fred went to the kitchen. Fred picked up the milk. Fred travelled to the office.
* Where is the milk ?
  * office
* Where does milk come from ? 
  * milk come from cow
* What is a cow a type of ? 
  * cow be female of cattle
* Where are cattle found ? 
  * cattle farm become widespread in brazil
* What does milk taste like ? 
  * milk taste like milk
* What does milk go well with ?
  * milk go with coffee
* Where was Fred before the office ? 
  * kitchen
  